In [22]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

import PIL
from PIL import Image

In [34]:
# resizeing images
f = r'C:\Users\setr7\Documents\Jupyter_notebook\Untitled Folder\Images_new\resize'
for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img)
    img = img.resize((256,256))
    img.save(f_img)

In [35]:
# Get images
#Images_new/Train/
images = []
for filename in os.listdir('Images_new/resize/'):
    images.append(img_to_array(load_img('Images_new/resize/'+filename)))
images = np.array(images, dtype=float)

# Set up train and test data
split = int(0.95*len(images))
images_train = images[:split]

# 1.0/255 -> 24 bit RGB color, [0-255] for each color channel -> 16.7m combinations -> human can recognize 2-10m 
images_train = 1.0/255*images_train

In [36]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [37]:
# Image datagenerator, adjust setting for img
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(images_train, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        # extract b/w layer for x_batch, two color layers for y_batch
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model  -- tensorboard not required ??    
tensorboard = TensorBoard(log_dir="output/first_run")
#callbacks=[tensorboard]
model.fit_generator(image_a_b_gen(batch_size), epochs=100, steps_per_epoch=10)

Epoch 1/100
10/10 [==============================] - 24s 2s/step - loss: 0.8926
Epoch 2/100
10/10 [==============================] - 27s 3s/step - loss: 0.3983
Epoch 3/100
10/10 [==============================] - 26s 3s/step - loss: 0.0123
Epoch 4/100
10/10 [==============================] - 29s 3s/step - loss: 0.0145
Epoch 5/100
10/10 [==============================] - 30s 3s/step - loss: 0.0135
Epoch 6/100
10/10 [==============================] - 30s 3s/step - loss: 0.0153
Epoch 7/100
10/10 [==============================] - 30s 3s/step - loss: 0.0139
Epoch 8/100
10/10 [==============================] - 30s 3s/step - loss: 0.0130
Epoch 9/100
10/10 [==============================] - 30s 3s/step - loss: 0.0156
Epoch 10/100
10/10 [==============================] - 30s 3s/step - loss: 0.0152
Epoch 11/100
10/10 [==============================] - 32s 3s/step - loss: 0.0121
Epoch 12/100
10/10 [==============================] - 32s 3s/step - loss: 0.0130
Epoch 13/100
10/10 [=================

In [38]:
# Save model
model_json = model.to_json()
with open("modelNew_resized2.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("modelNew_resized2.h5")

In [39]:

# Test images 
# Lab color, L -> grayscale layer, a b -> color layers (green-red, blue-yellow)
Xtest = rgb2lab(1.0/255*images[split:])[:,:,:,0]  # [:,:,:,0] -> select grayscale layer
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*images[split:])[:,:,:,1:] # [:,:,:,1] -> select color layer

# ab spectrum [-128,128] .. divide by 128 to get [-1,1]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

90/90 [==============================] - 5s 54ms/step
0.007139213693638642


In [40]:
# colorizer section 
# init array and add img to color
images_to_color = []
for filename in os.listdir('Images_new/Test/'):
    images_to_color.append(img_to_array(load_img('images_new/Test/'+filename)))

# covert color space, reshape img
images_to_color = np.array(images_to_color, dtype=float)
images_to_color = rgb2lab(1.0/255*images_to_color)[:,:,:,0]
images_to_color = images_to_color.reshape(images_to_color.shape+(1,))

# Test model with predict function
output = model.predict(images_to_color)

# covert color value back from [-1,1] to [-128,128] which is Lab color spectrum
output = output * 128

# Output colorizations
for i in range(len(output)):
    
    # make empty 256x256 with 3 layera
    cur = np.zeros((256, 256, 3))
    # copy grayscale layer to new obj
    cur[:,:,0] = images_to_color[i][:,:,0]
    # copy color layer to new obj
    cur[:,:,1:] = output[i]
    # save img in directory
    imsave("Images_new/Result/img_"+str(i)+".png", lab2rgb(cur))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
